In [1]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
import seaborn as sns
sns.set()
import matplotlib.pyplot as plt
import statsmodels.api as sm
import itertools

In [2]:
df=pd.read_csv("gold_rate_history.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'gold_rate_history.csv'

In [ ]:
pd.options.display.max_columns=None
pd.options.display.max_rows=None

# data preprocessing

In [ ]:
df.shape

In [ ]:
df.head(5000)

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.info()

In [ ]:
df['Date'].min(),df['Date'].max()

In [ ]:
df=df.groupby('Date')['Pure Gold (24 k)','Standard Gold (22 K)'].sum().reset_index()

In [ ]:
df['Date']=pd.to_datetime(df['Date'])
df.set_index('Date',inplace=True)
df.index

In [ ]:
df.head()

In [ ]:
y=df['Pure Gold (24 k)'].resample('MS').mean()

In [ ]:
y

In [ ]:
y.isnull().sum()

In [ ]:
y.plot(figsize=(15,6))
plt.show()

In [ ]:
d=df['Standard Gold (22 K)'].resample('MS').mean()

In [ ]:
d

In [ ]:
d.isnull().sum()

In [ ]:
d.plot(figsize=(15,6))
plt.show()

In [ ]:
from pylab import rcParams

rcParams['figure.figsize']=12,8

decomposition=sm.tsa.seasonal_decompose(y,model='additive')

fig=decomposition.plot()

plt.show()

In [ ]:
from pylab import rcParams

rcParams['figure.figsize']=12,8

decomposition=sm.tsa.seasonal_decompose(d,model='additive')

fig=decomposition.plot()

plt.show()

In [ ]:
from statsmodels.tsa.stattools import adfuller

In [ ]:
from pandas import Series
result=adfuller(y)
print('ADF Statistic: %f' % result[0])
print('P-value: %f' % result[1])
print('Critical Values:')
for key,value in result[4].items():
    print('\t%s: %.3f' % (key,value))

In [ ]:
from pandas import Series
result=adfuller(d)
print('ADF Statistic: %f' % result[0])
print('P-value: %f' % result[1])
print('Critical Values:')
for key,value in result[4].items():
    print('\t%s: %.3f' % (key,value))

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
decomposition = seasonal_decompose(y)

plt.plot(y,label = "Original")
plt.legend(loc = "best")

trend = decomposition.trend
plt.show()
plt.plot(trend, label = "Trend")
plt.legend(loc = "best")

seasonal = decomposition.seasonal
plt.show()
plt.plot(seasonal, label = "seasonal")
plt.legend(loc = "best")

residual = decomposition.resid
plt.show()
plt.plot(residual, label = "residual")
plt.legend(loc = "best")

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
decomposition = seasonal_decompose(d)

plt.plot(y,label = "Original")
plt.legend(loc = "best")

trend = decomposition.trend
plt.show()
plt.plot(trend, label = "Trend")
plt.legend(loc = "best")

seasonal = decomposition.seasonal
plt.show()
plt.plot(seasonal, label = "seasonal")
plt.legend(loc = "best")

residual = decomposition.resid
plt.show()
plt.plot(residual, label = "residual")
plt.legend(loc = "best")

In [ ]:
# ARIMA.

p = d = q = range(0,2)
pdq = list(itertools.product(p,d,q))
seasonal_pdq = [(x[0],x[1],x[2],12)for x in list(itertools.product(p,d,q))]
print("Examples of parameter combinations for seasonal ARIMA...")
print("SARIMA: {} x {}".format(pdq[1],seasonal_pdq[1]))
print("SARIMA: {} x {}".format(pdq[1],seasonal_pdq[2]))
print("SARIMA: {} x {}".format(pdq[1],seasonal_pdq[3]))
print("SARIMA: {} x {}".format(pdq[1],seasonal_pdq[4]))


In [ ]:
p

In [ ]:
from pylab import rcParams
for param in pdq:
    for param_seasonal in seasonal_pdq:
        try:
            mod = sm.tsa.statespace.SARIMAX(y, order = param,
seasonal_order=param_seasonal,                                          
enforce_stationarity = False,
enforce_invertibility=False)
            results=mod.fit()
            print('ARIMA{}x{}12 - ATC:{}'.format(param,param_seasonal,results.aic))
        except:
            continue

In [ ]:
df=sm.tsa.statespace.SARIMAX(y,order=(1,1,1),seasonal_order=(1,1,1,12),enforce_invertibility=False)
results=df.fit()
print(results.summary().tables[1])

In [ ]:
results.plot_diagnostics(figsize=(16,8))
plt.show()

In [ ]:
pred = results.get_prediction(start = pd.to_datetime("2008-01-01"),dynamis=False)
pred_ci = pred.conf_int()
ax = y['2008':].plot(label = "observed")
pred.predicted_mean.plot(ax=ax,label="one - step ahead Forecast",alpha=.7,figsize=(14,7))
ax.fill_between(pred_ci.index,
               pred_ci.iloc[:,0],
               pred_ci.iloc[:,1],color="k",alpha=.2)
ax.set_xlabel("Date")
ax.set_ylabel("pure gold 24k")
plt.legend()
plt.show()


In [ ]:
pred_uc = results.get_forecast(steps=13)
pred_ci = pred_uc.conf_int()
ax = y.plot(label = "observed",figsize=(14,7))
ax.fill_between(pred_ci.index,
               pred_ci.iloc[:,0],
               pred_ci.iloc[:,1],color="k",alpha=.25)
ax.set_xlabel("Date")
ax.set_ylabel("pure gold 24k")
print(pred_ci)
plt.legend()
plt.show()

In [ ]:
y_forecasted = pred.predicted_mean
y_truth = y["2008-01-01":]

mse = ((y_forecasted - y_truth) ** 2).mean()
print("the mse of out forecasts is {}".format(round(mse,2)))

In [ ]:
print(" the Root mean Squared error of our forecaste is {}".format(round(np.sqrt(mse),2)))

In [ ]:
pred_uc=results.get_forecast(steps=100)
pred_ci=pred_uc.conf_int()
ax=y.plot(label="observed",figsize=(14,7))
pred_uc.predicted_mean.plot(ax=ax,label="Forecast")
ax.fill_between(pred_ci.index,
               pred_ci.iloc[:,0],
               pred_ci.iloc[:,1],color="k",alpha=.25)
ax.set_xlabel("Date")
ax.set_ylabel("standard gold 22k")
print(pred_ci)
plt.legend()
plt.show()